In [ ]:
import openai
from PIL import Image, ImageDraw, ImageFont
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path


In [ ]:
openai.api_key = "OPENAI_KEY"


In [ ]:
def collect_user_inputs():
    """Collect comprehensive user inputs for the sports event trip planning"""
    print("\n===== SPORTS EVENT TRIP PLANNER =====")
    print("Let's create your personalized sports event journey roadmap!\n")

    start_date = input("Enter trip start date (YYYY-MM-DD, e.g., 2025-03-31): ")
    try:
        start_date_obj = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    except ValueError:
        print("Invalid start date format. Using today's date as default.")
        start_date_obj = datetime.datetime.now()
        start_date = start_date_obj.strftime('%Y-%m-%d')

    end_date = input("Enter trip end date (YYYY-MM-DD, e.g., 2025-04-02): ")
    try:
        end_date_obj = datetime.datetime.strptime(end_date, '%Y-%m-%d')
        if end_date_obj < start_date_obj:
            raise ValueError("End date must be after start date.")
    except ValueError as e:
        print(f"Invalid end date: {e}. Using 3 days from start date as default.")
        end_date_obj = start_date_obj + datetime.timedelta(days=3)
        end_date = end_date_obj.strftime('%Y-%m-%d')

    duration = (end_date_obj - start_date_obj).days + 1

    location = input("Enter event city and country (e.g., 'Riyadh, Saudi Arabia'): ")
    event_type = input("What type of sports event are you attending? (e.g., football, F1, tennis): ")
    event_name = input("Enter specific event name if known (e.g., 'FIFA World Cup', 'Saudi Grand Prix'): ")

    transport_mode = input("How will you arrive at your destination? (e.g., flight, train, drive): ")
    accommodation = input("Where will you be staying? (e.g., hotel name, Airbnb, staying with friends): ")
    accommodation_location = input("In which area/neighborhood is your accommodation? (e.g., downtown, near stadium): ")

    has_ticket = input("Do you have tickets for any events already? (yes/no): ").lower() == 'yes'
    ticket_details = []
    if has_ticket:
        num_tickets = int(input("How many different event tickets do you have? "))
        for i in range(num_tickets):
            print(f"\nTicket #{i+1}:")
            event = input("  Event name: ")
            date = input("  Date (YYYY-MM-DD): ")
            time = input("  Time (e.g., 19:30): ")
            venue = input("  Venue: ")
            seat = input("  Seat info (optional): ")
            ticket_details.append({"event": event, "date": date, "time": time, "venue": venue, "seat": seat})
    else:
        ticket_interest = input("Are you interested in getting tickets for any specific events? Please describe: ")

    print("\nPlease rate your interest in the following activities (1-5, where 5 is highest):")
    food_interest = int(input("Local cuisine exploration (1-5): ") or "3")
    sightseeing_interest = int(input("Sightseeing & tourism (1-5): ") or "3")
    fanzone_interest = int(input("Fan zones & meetups (1-5): ") or "3")
    shopping_interest = int(input("Shopping (1-5): ") or "2")
    nightlife_interest = int(input("Nightlife & entertainment (1-5): ") or "2")

    dietary_restrictions = input("Any dietary restrictions? ")
    mobility_limitations = input("Any mobility limitations to consider? ")
    budget_level = input("Budget level (economy, mid-range, luxury): ").lower() or "mid-range"
    special_requests = input("Any other special requests or preferences? ")

    return {
        "start_date": start_date,
        "end_date": end_date,
        "duration": duration,
        "location": location,
        "event_type": event_type,
        "event_name": event_name,
        "transport_mode": transport_mode,
        "accommodation": accommodation,
        "accommodation_location": accommodation_location,
        "has_ticket": has_ticket,
        "ticket_details": ticket_details if has_ticket else None,
        "ticket_interest": ticket_interest if not has_ticket else None,
        "preferences": {
            "food": food_interest,
            "sightseeing": sightseeing_interest,
            "fanzones": fanzone_interest,
            "shopping": shopping_interest,
            "nightlife": nightlife_interest
        },
        "dietary_restrictions": dietary_restrictions,
        "mobility_limitations": mobility_limitations,
        "budget_level": budget_level,
        "special_requests": special_requests
    }

In [ ]:

def generate_itinerary(inputs):
    """Generate detailed itinerary with explicit TIME field included"""
    ticket_info = "N/A"
    if inputs["has_ticket"] and inputs["ticket_details"]:
        ticket_info = "\n".join([
            f"- Event: {t['event']}, Date: {t['date']}, Time: {t['time']}, Venue: {t['venue']}, Seat: {t['seat']}"
            for t in inputs["ticket_details"]
        ])

    preference_ratings = []
    for pref, rating in inputs["preferences"].items():
        stars = "★" * rating + "☆" * (5 - rating)
        preference_ratings.append(f"{pref.capitalize()}: {stars} ({rating}/5)")
    preference_info = "\n".join(preference_ratings)

    prompt = f"""
Create a detailed sports event roadmap for a fan with the following details:

BASIC INFORMATION:
- Trip dates: {inputs['start_date']} to {inputs['end_date']} ({inputs['duration']} days)
- Location: {inputs['location']}
- Event type: {inputs['event_type']}
- Event name: {inputs['event_name']}

LOGISTICS:
- Transport mode: {inputs['transport_mode']}
- Accommodation: {inputs['accommodation']} at {inputs['accommodation_location']}

TICKETS:
- Has tickets: {'Yes' if inputs['has_ticket'] else 'No'}
- Ticket details: {ticket_info}
{f"- Ticket interests: {inputs['ticket_interest']}" if not inputs['has_ticket'] else ""}

PREFERENCES:
{preference_info}
- Dietary restrictions: {inputs['dietary_restrictions'] or 'None'}
- Mobility limitations: {inputs['mobility_limitations'] or 'None'}
- Budget level: {inputs['budget_level']}
- Special requests: {inputs['special_requests'] or 'None'}

INSTRUCTIONS:
1. Create a detailed daily itinerary for the trip duration.
2. Include arrival and departure logistics.
3. Incorporate the sports event(s) and mix in other activities based on preferences.
4. Assign approximate times where relevant (e.g., '14:00', '19:30').
5. Keep descriptions concise but informative.

Format each entry as:
NUMBER||EVENT_NAME||EVENT_DESCRIPTION||CATEGORY||DATE||TIME

Categories: SPORT, SHOPPING, FOOD, SIGHTSEEING, EVENT, TRANSPORT, ENTERTAINMENT, FAN_EXPERIENCE, ACCOMMODATION, OTHER

Example:
1||Arrival in Riyadh||Flight landing at King Khalid International||TRANSPORT||2025-03-31||14:00
2||Saudi Grand Prix||F1 race at Jeddah Corniche Circuit||SPORT||2025-04-01||19:30
"""

    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1500
    )
    return response.choices[0].message.content

In [ ]:
def generate_roadmap_visual(num_nodes=5, node_labels=None, event_details=None, filename="roadmap_no_text.png"):
    """
    Generates a horizontal roadmap visualization.
      - Nodes are aligned horizontally.
      - Alternating curved dashed arcs connect the nodes.
      - Each node ends with a small numbered circle.
      - If provided, node_labels (list of strings) are shown relative to the circle:
           * For nodes where the time is drawn above, event details appear in a box below.
           * For nodes where the time is drawn below, event details appear in a box above.
    
    Args:
        num_nodes (int): Number of nodes.
        node_labels (list of str, optional): Labels to display (e.g., TIME).
        event_details (list of str, optional): Detailed event text (e.g., "Lunch at XYZ\nLocation") to show in a box.
        filename (str): Where to save the image.
    """
    # --- Configuration ---
    node_radius = 0.32
    x_spacing = 2.5  # increased spacing for clarity
    arc_height = 0.5
    small_circle_radius = 0.1  # Increased by 30% from 0.07
    counter_line_offset = 0.2
    node_colors = ['#6053D3', '#292263']
    arc_line_color = 'black'
    line_style = '--'
    background_color = '#E0E0E0'
    number_color = 'white'
    number_fontsize = 12   # larger font for node numbers
    label_fontsize = 10    # larger font for time labels
    detail_fontsize = 12   # larger font for event detail boxes
    label_color = 'black'
    
    # --- Calculations ---
    main_node_y = 0
    x_coords = np.arange(num_nodes) * x_spacing
    # For visual clarity, we won't draw extra y_coords for main node.
    
    # Determine positions for the small numbered circles (for the connecting dashed line)
    y_coords_small = []
    for i in range(num_nodes):
        dist_to_small_center = node_radius + counter_line_offset + small_circle_radius
        # We'll keep the alternating offset for visual flair:
        if i % 2 == 0:  # for nodes 1,3,5,... (even index) we draw the small circle slightly lower
            small_y = main_node_y - dist_to_small_center + 0.1 * dist_to_small_center
        else:           # for nodes 2,4,6,... (odd index) we draw the small circle slightly higher
            small_y = main_node_y + dist_to_small_center - 0.1 * dist_to_small_center
        y_coords_small.append(small_y)
    y_coords_small = np.array(y_coords_small)
    
    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(4 * num_nodes, 8))
    fig.patch.set_facecolor(background_color)
    ax.set_facecolor(background_color)
    
    # Draw alternating arcs between nodes
    for i in range(num_nodes - 1):
        x1 = x_coords[i]
        x2 = x_coords[i+1]
        arc_center_x = (x1 + x2) / 2
        if i % 2 == 0:
            theta1, theta2 = 0, 180
        else:
            theta1, theta2 = 180, 360
        arc = patches.Arc((arc_center_x, main_node_y), width=x_spacing, height=arc_height,
                          angle=0, theta1=theta1, theta2=theta2,
                          color=arc_line_color, linestyle=line_style, lw=3, zorder=1)
        ax.add_patch(arc)
    
    # Draw nodes, dashed lines, numbered circles, and labels (time and event details)
    for i in range(num_nodes):
        node_color = node_colors[0] if i % 2 == 0 else node_colors[1]
        x = x_coords[i]
        small_y = y_coords_small[i]
        
        # Draw vertical dashed line from main node to small circle
        ax.plot([x, x], [main_node_y, small_y],
                linestyle=line_style, color=node_color, lw=2, zorder=1)
        # Draw main node (large circle)
        main_circle = plt.Circle((x, main_node_y), node_radius, color=node_color, zorder=2)
        ax.add_patch(main_circle)
        # Draw small numbered circle
        small_circle = plt.Circle((x, small_y), small_circle_radius,
                                  color=node_color, edgecolor=node_color,
                                  lw=2, zorder=3)
        ax.add_patch(small_circle)
        # Add node number
        ax.text(x, small_y, str(i + 1), ha='center', va='center',
                fontsize=number_fontsize, color=number_color, zorder=4)
        
        # Draw the time label (if provided) near the main node.
        # For even-indexed nodes (nodes 1,3,5,...), we draw time label above the circle.
        # For odd-indexed nodes (nodes 2,4,6,...), we draw time label below the circle.
        if node_labels and i < len(node_labels):
            offset = 0.2
            if i % 2 == 0:
                # Even-indexed: time label above
                time_y = main_node_y + node_radius + offset
                va_time = 'bottom'
            else:
                # Odd-indexed: time label below
                time_y = main_node_y - node_radius - offset
                va_time = 'top'
            ax.text(x, time_y, node_labels[i],
                    ha='center', va=va_time, fontsize=label_fontsize,
                    color=label_color, zorder=5)
        
        # Draw event details in a text box on the opposite side of the node from the time label.
        if event_details and i < len(event_details):
            box_offset = 0.6  # extra offset for the box
            bbox_props = dict(boxstyle="round,pad=0.5", fc="white", ec=node_color, alpha=0.9)
            if i % 2 == 0:
                # For even-indexed nodes (time label above), draw event box below.
                event_y = main_node_y - node_radius - box_offset
                va_event = 'top'
            else:
                # For odd-indexed nodes (time label below), draw event box above.
                event_y = main_node_y + node_radius + box_offset
                va_event = 'bottom'
            ax.text(x, event_y, event_details[i],
                    ha='center', va=va_event, fontsize=detail_fontsize,
                    color=label_color, bbox=bbox_props, zorder=6)
    
    # --- Appearance ---
    ax.set_aspect('equal', adjustable='box')
    padding_x = x_spacing * 0.8
    # Set vertical limits to leave room for event details boxes
    ax.set_xlim(x_coords[0] - padding_x, x_coords[-1] + padding_x)
    ax.set_ylim(main_node_y - 2.5, main_node_y + 2.5)
    ax.axis('off')
    plt.tight_layout()
    
    # Create directory if needed
    dir_name = os.path.dirname(filename)
    if dir_name:
        os.makedirs(dir_name, exist_ok=True)
    
    plt.savefig(filename, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
    plt.close(fig)
    print(f"Roadmap visual saved as '{filename}'")

In [ ]:
def create_day_roadmap(events, day_number, formatted_date, filename):
    """
    Creates a single-day roadmap image.
    Each node represents an event.
    The TIME is shown as the node label and the event details (name and location)
    appear in a text box on the opposite side.
    A day title (with the date) is drawn above the roadmap.
    """
    num_nodes = len(events)
    if num_nodes == 0:
        print(f"No events for day {day_number}, skipping.")
        return
    
    # Build node labels from the event time and event details as "Event Name\nLocation"
    node_labels = [ev["time"] if ev["time"] else "" for ev in events]
    event_details = [f"{ev['activity']}\n{ev['location']}" for ev in events]
    
    # Create the roadmap image for the day (save temporarily)
    temp_filename = f"roadmap_temp_day_{day_number}.png"
    generate_roadmap_visual(num_nodes=num_nodes, node_labels=node_labels, event_details=event_details, filename=temp_filename)
    
    # Open the generated roadmap image and add a day header (date)
    day_img = Image.open(temp_filename)
    width, height = day_img.size
    header_height = 80
    new_img = Image.new("RGB", (width, height + header_height), color='#E0E0E0')
    draw = ImageDraw.Draw(new_img)
    
    try:
        # Increased font size from 28 to 36 (approx 30% larger)
        header_font = ImageFont.truetype("arial.ttf", 400)
    except OSError:
        header_font = ImageFont.load_default()
    
    header_text = f"Day {day_number} - {formatted_date}"
    header_bbox = draw.textbbox((0, 0), header_text, font=header_font)
    header_width = header_bbox[2] - header_bbox[0]
    header_x = (width - header_width) // 2
    draw.text((header_x, 20), header_text, font=header_font, fill='#6053D3')
    
    new_img.paste(day_img, (0, header_height))
    new_img.save(filename)
    os.remove(temp_filename)
    print(f"Day {day_number} roadmap saved as '{filename}'")

In [ ]:
def combine_day_roadmaps(num_days, user_inputs, output_filename):
    """
    Combines individual day roadmap images into one final image.
    A common header with the overall event title and date range is added.
    """
    day_images = []
    for i in range(1, num_days + 1):
        day_file = f"roadmap/roadmap_day_{i}.png"
        if os.path.exists(day_file):
            day_images.append(Image.open(day_file))
    
    if not day_images:
        print("No day roadmaps found to combine.")
        return
    
    max_width = max(img.width for img in day_images)
    total_height = sum(img.height for img in day_images)
    header_height = 200
    combined = Image.new('RGB', (max_width, total_height + header_height), color='#E0E0E0')
    draw = ImageDraw.Draw(combined)
    
    try:
        # Increased font sizes by approximately 30%
        title_font = ImageFont.truetype("arial.ttf", 500)  # from 48
        subtitle_font = ImageFont.truetype("arial.ttf", 450)  # from 32
        info_font = ImageFont.truetype("arial.ttf", 420)  # from 24
    except OSError:
        title_font = ImageFont.load_default()
        subtitle_font = ImageFont.load_default()
        info_font = ImageFont.load_default()
    
    title = "Sports Event Roadmap"
    subtitle = f"{user_inputs['event_name']} in {user_inputs['location']}"
    date_range = f"{user_inputs['start_date']} to {user_inputs['end_date']}  •  {user_inputs['duration']} days"
    
    title_bbox = draw.textbbox((0, 0), title, font=title_font)
    title_width = title_bbox[2] - title_bbox[0]
    title_x = (max_width - title_width) // 2
    
    subtitle_bbox = draw.textbbox((0, 0), subtitle, font=subtitle_font)
    subtitle_width = subtitle_bbox[2] - subtitle_bbox[0]
    subtitle_x = (max_width - subtitle_width) // 2
    
    info_bbox = draw.textbbox((0, 0), date_range, font=info_font)
    info_width = info_bbox[2] - info_bbox[0]
    info_x = (max_width - info_width) // 2
    
    draw.text((title_x, 30), title, font=title_font, fill='#6053D3')
    draw.text((subtitle_x, 100), subtitle, font=subtitle_font, fill='#292263')
    draw.text((info_x, 150), date_range, font=info_font, fill='#555555')
    
    y_offset = header_height
    for img in day_images:
        x_offset = (max_width - img.width) // 2
        combined.paste(img, (x_offset, y_offset))
        y_offset += img.height
    
    combined.save(output_filename)
    print(f"Final combined roadmap saved as '{output_filename}'")
    
    # Optionally, remove individual day images
    for i in range(1, num_days + 1):
        try:
            os.remove(f"roadmap/roadmap_day_{i}.png")
        except Exception:
            pass

In [ ]:
def create_improved_roadmap_image(itinerary_text, user_inputs, output_filename="sports_roadmap.png"):
    """
    Parses the itinerary text (each entry as:
    NUMBER||EVENT_NAME||EVENT_DESCRIPTION||CATEGORY||DATE||TIME),
    groups events by day, creates a single horizontal roadmap for each day,
    and finally combines all day roadmaps into one final image.
    """
    Path("roadmap").mkdir(exist_ok=True)
    
    # Parse itinerary lines
    lines = [line.strip() for line in itinerary_text.split('\n') if line.strip()]
    events = []
    for line in lines:
        parts = line.split("||")
        if len(parts) >= 6:
            event = {
                "event_num": parts[0].strip(),
                "activity": parts[1].strip(),
                "description": parts[2].strip(),
                "category": parts[3].strip(),
                "date": parts[4].strip(),
                "time": parts[5].strip() if parts[5].strip() else "TBD"
            }
            # Extract location hint from description if available
            location_parts = event["description"].split(',')
            event["location"] = location_parts[-1].strip() if len(location_parts) > 1 else "Venue TBD"
            events.append(event)
    
    if not events:
        print("No itinerary data found.")
        return
    
    # Group events by date
    grouped = {}
    for ev in events:
        grouped.setdefault(ev["date"], []).append(ev)
    
    try:
        sorted_dates = sorted(grouped.keys(), key=lambda d: datetime.datetime.strptime(d, '%Y-%m-%d'))
    except ValueError:
        sorted_dates = sorted(grouped.keys())
    
    # Create a roadmap image for each day
    for day_count, date_key in enumerate(sorted_dates, 1):
        day_events = grouped[date_key]
        try:
            date_obj = datetime.datetime.strptime(date_key, '%Y-%m-%d')
            formatted_date = date_obj.strftime('%A, %B %d, %Y')
        except ValueError:
            formatted_date = date_key
        day_filename = f"roadmap/roadmap_day_{day_count}.png"
        create_day_roadmap(day_events, day_count, formatted_date, day_filename)
    
    # Combine all day roadmaps into one final image
    combine_day_roadmaps(len(sorted_dates), user_inputs, output_filename)

In [ ]:
def main():
    try:
        openai.api_key = input("Enter your OpenAI API key: ")
        user_inputs = collect_user_inputs()
        print("\nGenerating your sports event roadmap...")
        itinerary = generate_itinerary(user_inputs)
        print("\nGenerated Itinerary Data:")
        print(itinerary)
        print("\nCreating your high-quality visual roadmap...")
        create_improved_roadmap_image(itinerary, user_inputs)
        print("\nYour sports event roadmap has been created successfully!")
        show_image = input("\nWould you like to view the roadmap now? (yes/no): ").lower() == 'yes'
        if show_image:
            try:
                Image.open("sports_roadmap.png").show()
            except Exception as e:
                print(f"Couldn't display the image: {e}")
                print("Please open the sports_roadmap.png file manually.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
if __name__ == "__main__":
    main()